# Data Wrangling Part 2

In [1]:
# importing necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import fuzzywuzzy
from IPython.display import display

In [2]:
# setting options to show all column and row names
pd.options.display.max_columns=None
pd.options.display.max_rows=None

In [3]:
# reading csv files
act = pd.read_csv('act_data.csv')
census = pd.read_csv('census_data.csv', index_col = [0])
gsid = pd.read_csv('school_gsid.csv', index_col = [0])

In [4]:
# adding county names to gsid data using the city names in the excel file
df = pd.read_excel("cities in counties.xlsx")
county = list(df['County'])
city = list(df['City'])
city_cnty = dict(zip(city, county))
gsid['County'] = gsid['City'].map(city_cnty) 

So now every school name in gsid data also have a county name which  will help when merging the dataframes using these two columns. 

In [5]:
# formatting the school names so that the names match with eachother in three datasets
gsid['Sname'] = gsid['Sname'].str.strip('School').str.strip()
gsid['City'] = gsid['City'].str.replace('-', ' ').str.strip()
census['Sname'] = census['Sname'].str.strip('School').str.strip()

In [6]:
# changing the format of floats to display only two digits
pd.options.display.float_format = '{:,.2f}'.format

In [7]:
act=act[act['Rtype'] == 'S']
act.head()

,Year,Rtype,Sname,Dname,Cname,Enroll12,NumTstTakr,AvgScrEng,AvgScrRead,AvgScrMath,AvgScrSci,NumGE21,PctGE21
0,2014,S,Granada High,Livermore Valley Joint Unified,Alameda,467,108,26.00,26.00,26.00,26.00,99,91.67
1,2014,S,Livermore High,Livermore Valley Joint Unified,Alameda,438,74,25.00,25.00,25.00,24.00,59,79.73
3,2014,S,Newark Memorial High,Newark Unified,Alameda,418,54,22.00,23.00,23.00,22.00,34,62.96
5,2014,S,James Logan High,New Haven Unified,Alameda,943,242,21.00,22.00,24.00,22.00,150,61.98
7,2014,S,Oakland Unity High,Oakland Unified,Alameda,67,34,13.00,16.00,18.00,16.00,4,11.76


In [8]:
act.shape

(5371, 13)

In [9]:
census.head()

,ActSat Number Participation,ApCourse Participation,ApCourse Passed,Asian,"Black, non-Hispanic",FreeAnd Reduced PriceLunch,Hispanic,Id,Multiracial,Native American or Native Alaskan,Native Hawaiian or Other Pacific Islander,Percent Teachers InFirst SecondYear,Plan504,Sname,"White, non-Hispanic",Year,index
0,1071,46.80,885,45.63,6.52,18.16,11.87,1,3.96,0.17,0.76,7.90,1.50,Alameda High,31.08,2016,nan
1,25,0.00,885,70.29,4.00,30.29,8.57,14052,2.29,0.00,2.29,12.50,1.20,Alameda Science And Technology Institute,12.57,2016,nan
2,25,0.00,885,0.00,0.00,30.29,0.00,17140,100.00,0.00,0.00,12.50,1.20,"Applied Scholastics Academy, East Bay",0.00,2016,nan
3,178,0.00,885,35.63,17.51,40.34,16.69,6,6.66,0.30,1.42,1.80,2.00,Encinal Junior/Senior Hig,21.78,2016,nan
4,4,0.00,885,10.26,18.80,46.15,39.32,9,7.69,1.71,2.56,10.00,3.90,Island High (Continuation),19.66,2016,nan


In [10]:
census.shape

(2410, 17)

In [11]:
gsid.head()

,City,Sname,Type,gsId,Lat,Lon,Enrollment,gsRating,ParentRating,County
0,Alameda,Alameda Community Learning Center,charter,11902,37.78,-122.29,360.00,8.00,4.00,Alameda
1,Alameda,Alameda High,public,1,37.76,-122.25,"1,853.00",9.00,4.00,Alameda
2,Alameda,Alameda Science And Technology Institute,public,14052,37.78,-122.28,175.00,10.00,4.00,Alameda
3,Alameda,"Applied Scholastics Academy, East Bay",private,17140,37.77,-122.25,13.00,nan,5.00,Alameda
4,Alameda,Children's Learning Center,private,10045,37.77,-122.25,nan,nan,5.00,Alameda


In [12]:
gsid.shape

(4271, 10)

## Merging census and gsid data

In [13]:
# merging the census and gsid using the id columns
census_id = pd.merge(census, gsid, left_on='Id', right_on='gsId', how ='inner', suffixes = ('','_gsid'))

In [14]:
census_id.head()

,ActSat Number Participation,ApCourse Participation,ApCourse Passed,Asian,"Black, non-Hispanic",FreeAnd Reduced PriceLunch,Hispanic,Id,Multiracial,Native American or Native Alaskan,Native Hawaiian or Other Pacific Islander,Percent Teachers InFirst SecondYear,Plan504,Sname,"White, non-Hispanic",Year,index,City,Sname_gsid,Type,gsId,Lat,Lon,Enrollment,gsRating,ParentRating,County
0,1071,46.80,885,45.63,6.52,18.16,11.87,1,3.96,0.17,0.76,7.90,1.50,Alameda High,31.08,2016,nan,Alameda,Alameda High,public,1,37.76,-122.25,"1,853.00",9.00,4.00,Alameda
1,25,0.00,885,70.29,4.00,30.29,8.57,14052,2.29,0.00,2.29,12.50,1.20,Alameda Science And Technology Institute,12.57,2016,nan,Alameda,Alameda Science And Technology Institute,public,14052,37.78,-122.28,175.00,10.00,4.00,Alameda
2,25,0.00,885,0.00,0.00,30.29,0.00,17140,100.00,0.00,0.00,12.50,1.20,"Applied Scholastics Academy, East Bay",0.00,2016,nan,Alameda,"Applied Scholastics Academy, East Bay",private,17140,37.77,-122.25,13.00,nan,5.00,Alameda
3,178,0.00,885,35.63,17.51,40.34,16.69,6,6.66,0.30,1.42,1.80,2.00,Encinal Junior/Senior Hig,21.78,2016,nan,Alameda,Encinal Junior/Senior Hig,public,6,37.77,-122.29,"1,336.00",6.00,4.00,Alameda
4,4,0.00,885,10.26,18.80,46.15,39.32,9,7.69,1.71,2.56,10.00,3.90,Island High (Continuation),19.66,2016,nan,Alameda,Island High (Continuation),public,9,37.78,-122.28,117.00,nan,4.00,Alameda


In [15]:
census_id.shape

(2391, 27)

In [16]:
# removing redundant columns from the merged data
census_id = census_id.drop(['Id','Year','index','Sname_gsid'], axis = 1)

In [17]:
census_id.head()

,ActSat Number Participation,ApCourse Participation,ApCourse Passed,Asian,"Black, non-Hispanic",FreeAnd Reduced PriceLunch,Hispanic,Multiracial,Native American or Native Alaskan,Native Hawaiian or Other Pacific Islander,Percent Teachers InFirst SecondYear,Plan504,Sname,"White, non-Hispanic",City,Type,gsId,Lat,Lon,Enrollment,gsRating,ParentRating,County
0,1071,46.80,885,45.63,6.52,18.16,11.87,3.96,0.17,0.76,7.90,1.50,Alameda High,31.08,Alameda,public,1,37.76,-122.25,"1,853.00",9.00,4.00,Alameda
1,25,0.00,885,70.29,4.00,30.29,8.57,2.29,0.00,2.29,12.50,1.20,Alameda Science And Technology Institute,12.57,Alameda,public,14052,37.78,-122.28,175.00,10.00,4.00,Alameda
2,25,0.00,885,0.00,0.00,30.29,0.00,100.00,0.00,0.00,12.50,1.20,"Applied Scholastics Academy, East Bay",0.00,Alameda,private,17140,37.77,-122.25,13.00,nan,5.00,Alameda
3,178,0.00,885,35.63,17.51,40.34,16.69,6.66,0.30,1.42,1.80,2.00,Encinal Junior/Senior Hig,21.78,Alameda,public,6,37.77,-122.29,"1,336.00",6.00,4.00,Alameda
4,4,0.00,885,10.26,18.80,46.15,39.32,7.69,1.71,2.56,10.00,3.90,Island High (Continuation),19.66,Alameda,public,9,37.78,-122.28,117.00,nan,4.00,Alameda


## Merging act and census_id data

In [18]:
census_id.shape

(2391, 23)

In [19]:
len(act['Sname'])

5371

In [20]:
# importing the necessary package to be able to match the school names in both datasets.

from fuzzywuzzy import fuzz

def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        # Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if  (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
           
    return (max_name, max_score)


In the next step, I am trying to find the closest matches of the act school names with the ones in merged_gs data.
I created a list that consists of the school names in act data and their corresponding names in the merge_gs data using the function defined above which gives me the matches where the matching score is at least %81. 

In [21]:
# creating a dictionary for easy dataframe creation

dict_list = []
# iterating over act school names
for name in act['Sname']:
    
    # using the method to find best match setting a threshold here
    match = match_name(name, census_id['Sname'], 81)
    if match == True:
        continue
 
    # New dict for storing data
    dict_ = {}
    dict_.update({"actSname" : name})
    dict_.update({"mergedSname" : match[0]})
    dict_.update({"score" : match[1]})
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)
# Display results
merge_table.head()

,actSname,mergedSname,score
0,Granada High,Granada High,100
1,Livermore High,Livermore High,100
2,Newark Memorial High,Newark Memorial High,100
3,James Logan High,James Logan high s,88
4,Oakland Unity High,Oakland Unity High,100



Right now, I have a dictionary that contains all of the school names in the act data and their at least %81 matching names in the merge_gs data. By using this dictionary as map, I am adding the corresponding school names into my original act data to be able to merge the two data sets later on. 

In [22]:
Sname = list(merge_table['actSname'])
mergedSname = list(merge_table['mergedSname'])
Sname_mergedSname = dict(zip(Sname, mergedSname))
act['Sname2'] = act['Sname'].map(Sname_mergedSname) 

In [23]:
 act.head()

,Year,Rtype,Sname,Dname,Cname,Enroll12,NumTstTakr,AvgScrEng,AvgScrRead,AvgScrMath,AvgScrSci,NumGE21,PctGE21,Sname2
0,2014,S,Granada High,Livermore Valley Joint Unified,Alameda,467,108,26.00,26.00,26.00,26.00,99,91.67,Granada High
1,2014,S,Livermore High,Livermore Valley Joint Unified,Alameda,438,74,25.00,25.00,25.00,24.00,59,79.73,Livermore High
3,2014,S,Newark Memorial High,Newark Unified,Alameda,418,54,22.00,23.00,23.00,22.00,34,62.96,Newark Memorial High
5,2014,S,James Logan High,New Haven Unified,Alameda,943,242,21.00,22.00,24.00,22.00,150,61.98,James Logan high s
7,2014,S,Oakland Unity High,Oakland Unified,Alameda,67,34,13.00,16.00,18.00,16.00,4,11.76,Oakland Unity High


In [24]:
act.shape

(5371, 14)

 
In this step, I am merging the act data with the census_id data based on two columns which are the school names and the county names. I am making sure that even though the school names are same they also need to be in the same county, because there are several schools with the same name in different counties. 


In [25]:
final_df = pd.merge(act, census_id, left_on=['Sname2', 'Cname'], right_on=['Sname', 'County'],
                    how ='inner',suffixes = ('','3'))

In [26]:
final_df.head()

,Year,Rtype,Sname,Dname,Cname,Enroll12,NumTstTakr,AvgScrEng,AvgScrRead,AvgScrMath,AvgScrSci,NumGE21,PctGE21,Sname2,ActSat Number Participation,ApCourse Participation,ApCourse Passed,Asian,"Black, non-Hispanic",FreeAnd Reduced PriceLunch,Hispanic,Multiracial,Native American or Native Alaskan,Native Hawaiian or Other Pacific Islander,Percent Teachers InFirst SecondYear,Plan504,Sname3,"White, non-Hispanic",City,Type,gsId,Lat,Lon,Enrollment,gsRating,ParentRating,County
0,2014,S,Granada High,Livermore Valley Joint Unified,Alameda,467,108,26.00,26.00,26.00,26.00,99,91.67,Granada High,479,28.50,542,9.96,1.55,15.06,24.58,7.29,0.34,0.34,9.10,2.90,Granada High,55.95,Livermore,public,148,37.68,-121.79,"2,059.00",9.00,4.00,Alameda
1,2015,S,Granada High,Livermore Valley Joint Unified,Alameda,454,114,25.00,25.00,26.00,25.00,95,83.33,Granada High,479,28.50,542,9.96,1.55,15.06,24.58,7.29,0.34,0.34,9.10,2.90,Granada High,55.95,Livermore,public,148,37.68,-121.79,"2,059.00",9.00,4.00,Alameda
2,2016,S,Granada High,Livermore Valley Joint Unified,Alameda,473,157,26.00,26.00,26.00,25.00,139,88.54,Granada High,479,28.50,542,9.96,1.55,15.06,24.58,7.29,0.34,0.34,9.10,2.90,Granada High,55.95,Livermore,public,148,37.68,-121.79,"2,059.00",9.00,4.00,Alameda
3,2017,S,Granada High,Livermore Valley Joint Unified,Alameda,470,158,26.00,26.00,26.00,25.00,137,86.71,Granada High,479,28.50,542,9.96,1.55,15.06,24.58,7.29,0.34,0.34,9.10,2.90,Granada High,55.95,Livermore,public,148,37.68,-121.79,"2,059.00",9.00,4.00,Alameda
4,2018,S,Granada High,Livermore Valley Joint Unified,Alameda,513,148,25.00,26.00,26.00,25.00,126,85.14,Granada High,479,28.50,542,9.96,1.55,15.06,24.58,7.29,0.34,0.34,9.10,2.90,Granada High,55.95,Livermore,public,148,37.68,-121.79,"2,059.00",9.00,4.00,Alameda


In [27]:
final_df.shape

(4229, 37)

In [28]:
final_df.columns

Index(['Year', 'Rtype', 'Sname', 'Dname', 'Cname', 'Enroll12', 'NumTstTakr',
       'AvgScrEng', 'AvgScrRead', 'AvgScrMath', 'AvgScrSci', 'NumGE21',
       'PctGE21', 'Sname2', 'ActSat Number Participation',
       'ApCourse Participation', 'ApCourse Passed', 'Asian',
       'Black, non-Hispanic', 'FreeAnd Reduced PriceLunch', 'Hispanic',
       'Multiracial', 'Native American or Native Alaskan',
       'Native Hawaiian or Other Pacific Islander',
       'Percent Teachers InFirst SecondYear', 'Plan504', 'Sname3',
       'White, non-Hispanic', 'City', 'Type', 'gsId', 'Lat', 'Lon',
       'Enrollment', 'gsRating', 'ParentRating', 'County'],
      dtype='object')

In [29]:
# setting the  year  column as index
final_df = final_df.set_index(['Year'])
final_df.index = pd.to_datetime(final_df.index, format = '%Y', errors='ignore').year

In [30]:
# changing some of the column types
final_df['gsId'] = final_df['gsId'].astype(str)
final_df['Enrollment'] = final_df['Enrollment'].astype(int)

In [31]:
len(final_df['Sname'].unique())- len(final_df['Sname2'].unique())

34

Here, we understand that there are few school names which are matched to the same school name.

In [32]:
# removing the school names that causes duplicates in the final_df, these schools are incorrectly matched, 
# even though they don't have a corresponding name in the census data,  they are matched with other existing ones.
final_df = final_df[~final_df["Sname"].str.contains('NAVA College Preparatory')]
final_df = final_df[~final_df["Sname"].str.contains('North Hollywood Senior')]
final_df = final_df[~final_df["Sname"].str.contains('San Pedro Senior')]

In [33]:
# creating a dictionary to replace the correct school names for some mismatched ones 
corrections = {'Oakland Military Institute, College Prep':'Oakland Military Institute, College Preparatory Ac',
'Aspire Lionel Wilson College Preparatory':'Aspire Lionel Wilson College Preparatory Academy',
'Aspire Lionel Wilson College Preparatory':'Aspire Lionel Wilson College Preparatory Academy',
'Morningside High School':'Morningside High', 
'City Honors College Preparatory Charter':'City Honors College Preparatory Academy',
'Alliance Tennenbaum Family Technology Hi':'Alliance Tennenbaum Family Technology High',
'Los Angeles Leadership Primary Academy':'Los Angeles Leadership Academy',
'Alliance Renee and Meyer Luskin Academy':'Alliance Renee and Meyer Luskin Academy High',
'Augustus F. Hawkins High A Critical Desi':'Augustus F. Hawkins High A Critical Design and Gam',
'Madera South High':'Madera High', 
'California Academy of Mathematics and Sc':'California Academy of Mathematics and Science',
'View Park Preparatory Accelerated High':'ICEF View Park Preparatory Charter High',
'Alliance Judy Ivie Burton Technology Aca':'Alliance Judy Ivie Burton Technology Academy High',
'Academies of Education and Empowerment a':'Academies of Education and Empowerment at Carson H',
'Contreras Learning Center-School of Soci':'Contreras Learning Center-School of Social Justice',
'Village Academy High School at Indian Hi':'Village Academy High School at Indian Hill',
'Fremont Academy of Engineering and Desig':'Fremont Academy of Engineering and Design',
'Aspire Benjamin Holt College Preparatory':'Aspire Benjamin Holt College Preparatory Academy',
'Dr. T. J. Owens Gilroy Early College Aca':'Dr. T. J. Owens Gilroy Early College Academy',
'San JosŽ High':'San Jose High',
'Natomas Pacific Pathways Prep Middle':'Natomas Pacific Pathways Prep',
'Animo Ralph Bunche Charter High':'Animo Ralph Bunche High',
'Alliance Marc & Eva Stern Math and Scien':'Alliance Marc & Eva Stern Math and Science',
'Design Science Early College High':'Design Science Middle College High',
'Magnolia Science Academy 2': 'Magnolia Science Academy 3',
'Camino Nuevo High #2':'Camino Nuevo High No. 2',
'Birch High (Continuation)': 'Eric Birch High (Continuation)',
'Alain Leroy Locke College Prep Academy':'Alain Leroy Locke College Preparatory Academy',
'Alliance Cindy and Bill Simon Technology Academy H':'Alliance Cindy and Bill Simon Technology Academy', 
'Alliance Cindy and Bill Simon Technology':'Alliance Cindy and Bill Simon Technology Academy'}

final_df =final_df.replace({'Sname': corrections})

In [34]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4218 entries, 2014 to 2018
Data columns (total 36 columns):
Rtype                                        4218 non-null object
Sname                                        4218 non-null object
Dname                                        4218 non-null object
Cname                                        4218 non-null object
Enroll12                                     4218 non-null int64
NumTstTakr                                   4218 non-null int64
AvgScrEng                                    4218 non-null float64
AvgScrRead                                   4218 non-null float64
AvgScrMath                                   4218 non-null float64
AvgScrSci                                    4218 non-null float64
NumGE21                                      4218 non-null int64
PctGE21                                      4218 non-null float64
Sname2                                       4218 non-null object
ActSat Number Participation   

In [35]:
final_df.isnull().sum()

Rtype                                          0
Sname                                          0
Dname                                          0
Cname                                          0
Enroll12                                       0
NumTstTakr                                     0
AvgScrEng                                      0
AvgScrRead                                     0
AvgScrMath                                     0
AvgScrSci                                      0
NumGE21                                        0
PctGE21                                        0
Sname2                                         0
ActSat Number Participation                    0
ApCourse Participation                         0
ApCourse Passed                                0
Asian                                          0
Black, non-Hispanic                            0
FreeAnd Reduced PriceLunch                     0
Hispanic                                       0
Multiracial         

In [36]:
final_df['gsRating'].value_counts(dropna=False)

6.00     629
9.00     577
8.00     517
4.00     505
5.00     489
7.00     454
3.00     331
10.00    307
2.00     272
1.00     126
nan       11
Name: gsRating, dtype: int64

In [37]:
final_df.columns

Index(['Rtype', 'Sname', 'Dname', 'Cname', 'Enroll12', 'NumTstTakr',
       'AvgScrEng', 'AvgScrRead', 'AvgScrMath', 'AvgScrSci', 'NumGE21',
       'PctGE21', 'Sname2', 'ActSat Number Participation',
       'ApCourse Participation', 'ApCourse Passed', 'Asian',
       'Black, non-Hispanic', 'FreeAnd Reduced PriceLunch', 'Hispanic',
       'Multiracial', 'Native American or Native Alaskan',
       'Native Hawaiian or Other Pacific Islander',
       'Percent Teachers InFirst SecondYear', 'Plan504', 'Sname3',
       'White, non-Hispanic', 'City', 'Type', 'gsId', 'Lat', 'Lon',
       'Enrollment', 'gsRating', 'ParentRating', 'County'],
      dtype='object')

In [38]:
# dropping the redundant columns 
final_df = final_df.drop(['Rtype','Sname2','Sname3','Cname'], axis = 1)

In [39]:
final_df.columns

Index(['Sname', 'Dname', 'Enroll12', 'NumTstTakr', 'AvgScrEng', 'AvgScrRead',
       'AvgScrMath', 'AvgScrSci', 'NumGE21', 'PctGE21',
       'ActSat Number Participation', 'ApCourse Participation',
       'ApCourse Passed', 'Asian', 'Black, non-Hispanic',
       'FreeAnd Reduced PriceLunch', 'Hispanic', 'Multiracial',
       'Native American or Native Alaskan',
       'Native Hawaiian or Other Pacific Islander',
       'Percent Teachers InFirst SecondYear', 'Plan504', 'White, non-Hispanic',
       'City', 'Type', 'gsId', 'Lat', 'Lon', 'Enrollment', 'gsRating',
       'ParentRating', 'County'],
      dtype='object')

In [40]:
# ordering the column names
cols = ['Sname','gsId', 'Dname', 'County', 'City', 'Lat', 'Lon', 'Type','Enrollment','Enroll12',
       'Percent Teachers InFirst SecondYear', 'Plan504','FreeAnd Reduced PriceLunch',  
       'NumTstTakr', 'AvgScrEng','AvgScrRead', 'AvgScrMath', 'AvgScrSci', 'NumGE21', 'PctGE21', 
       'ActSat Number Participation','Asian', 'Black, non-Hispanic', 'Hispanic', 'Multiracial', 
       'Native American or Native Alaskan','Native Hawaiian or Other Pacific Islander', 
       'White, non-Hispanic', 'gsRating', 'ParentRating']

In [41]:
final_df = final_df[cols]

In [42]:
final_df.head()

,Sname,gsId,Dname,County,City,Lat,Lon,Type,Enrollment,Enroll12,Percent Teachers InFirst SecondYear,Plan504,FreeAnd Reduced PriceLunch,NumTstTakr,AvgScrEng,AvgScrRead,AvgScrMath,AvgScrSci,NumGE21,PctGE21,ActSat Number Participation,Asian,"Black, non-Hispanic",Hispanic,Multiracial,Native American or Native Alaskan,Native Hawaiian or Other Pacific Islander,"White, non-Hispanic",gsRating,ParentRating
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014,Granada High,148,Livermore Valley Joint Unified,Alameda,Livermore,37.68,-121.79,public,2059,467,9.10,2.90,15.06,108,26.00,26.00,26.00,26.00,99,91.67,479,9.96,1.55,24.58,7.29,0.34,0.34,55.95,9.00,4.00
2015,Granada High,148,Livermore Valley Joint Unified,Alameda,Livermore,37.68,-121.79,public,2059,454,9.10,2.90,15.06,114,25.00,25.00,26.00,25.00,95,83.33,479,9.96,1.55,24.58,7.29,0.34,0.34,55.95,9.00,4.00
2016,Granada High,148,Livermore Valley Joint Unified,Alameda,Livermore,37.68,-121.79,public,2059,473,9.10,2.90,15.06,157,26.00,26.00,26.00,25.00,139,88.54,479,9.96,1.55,24.58,7.29,0.34,0.34,55.95,9.00,4.00
2017,Granada High,148,Livermore Valley Joint Unified,Alameda,Livermore,37.68,-121.79,public,2059,470,9.10,2.90,15.06,158,26.00,26.00,26.00,25.00,137,86.71,479,9.96,1.55,24.58,7.29,0.34,0.34,55.95,9.00,4.00
2018,Granada High,148,Livermore Valley Joint Unified,Alameda,Livermore,37.68,-121.79,public,2059,513,9.10,2.90,15.06,148,25.00,26.00,26.00,25.00,126,85.14,479,9.96,1.55,24.58,7.29,0.34,0.34,55.95,9.00,4.00


In [43]:
final_df.describe()

,Lat,Lon,Enrollment,Enroll12,Percent Teachers InFirst SecondYear,Plan504,FreeAnd Reduced PriceLunch,NumTstTakr,AvgScrEng,AvgScrRead,AvgScrMath,AvgScrSci,NumGE21,PctGE21,ActSat Number Participation,Asian,"Black, non-Hispanic",Hispanic,Multiracial,Native American or Native Alaskan,Native Hawaiian or Other Pacific Islander,"White, non-Hispanic",gsRating,ParentRating
count,"4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,218.00","4,207.00","4,094.00"
mean,35.67,-119.53,"1,682.13",386.49,8.36,1.20,55.36,101.00,20.86,21.70,21.73,21.10,58.65,52.74,277.90,12.55,6.21,52.84,3.05,0.55,0.56,24.24,6.06,3.79
std,2.09,2.01,823.29,203.80,11.26,1.36,26.84,75.47,3.89,3.47,3.27,3.09,62.09,25.43,448.28,15.73,8.85,27.36,2.99,0.94,0.73,22.27,2.47,0.58
min,32.57,-124.16,70.00,0.00,0.00,0.00,0.68,11.00,8.00,11.00,14.00,11.00,0.00,0.00,0.00,0.00,0.00,1.59,0.00,0.00,0.00,0.00,1.00,2.00
25%,33.92,-121.62,"1,053.00",222.00,2.10,0.30,32.28,45.00,18.00,19.00,19.00,19.00,17.00,31.72,97.00,2.36,1.29,29.04,0.83,0.16,0.14,4.71,4.00,3.00
50%,34.53,-118.91,"1,743.00",397.00,5.30,0.80,58.43,83.00,21.00,22.00,21.00,21.00,36.00,54.36,206.00,6.76,2.83,53.33,2.13,0.33,0.33,17.61,6.00,4.00
75%,37.69,-117.88,"2,240.00",527.00,10.40,1.70,79.11,135.00,24.00,24.00,24.00,23.00,77.00,74.48,354.00,15.36,7.77,76.55,4.46,0.62,0.68,40.34,8.00,4.00
max,41.74,-114.61,"5,000.00","1,158.00",100.00,18.20,99.47,660.00,32.00,31.00,32.00,31.00,387.00,100.00,"6,081.00",91.78,95.41,100.00,27.65,12.54,5.67,88.36,10.00,5.00


In [44]:
# checking the max value of the enrollment column to see  if they  are  outliers.  
final_df[final_df['Enrollment']>4500]

,Sname,gsId,Dname,County,City,Lat,Lon,Type,Enrollment,Enroll12,Percent Teachers InFirst SecondYear,Plan504,FreeAnd Reduced PriceLunch,NumTstTakr,AvgScrEng,AvgScrRead,AvgScrMath,AvgScrSci,NumGE21,PctGE21,ActSat Number Participation,Asian,"Black, non-Hispanic",Hispanic,Multiracial,Native American or Native Alaskan,Native Hawaiian or Other Pacific Islander,"White, non-Hispanic",gsRating,ParentRating
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014,Paramount High,2772,Paramount Unified,Los Angeles,Paramount,33.90,-118.15,public,4794,1112,6.40,0.10,91.24,158,17.00,19.00,20.00,18.00,47,29.75,354,1.48,8.16,87.76,0.73,0.02,0.83,1.02,4.00,3.00
2015,Paramount High,2772,Paramount Unified,Los Angeles,Paramount,33.90,-118.15,public,4794,1118,6.40,0.10,91.24,129,17.00,19.00,19.00,19.00,27,20.93,354,1.48,8.16,87.76,0.73,0.02,0.83,1.02,4.00,3.00
2016,Paramount High,2772,Paramount Unified,Los Angeles,Paramount,33.90,-118.15,public,4794,1110,6.40,0.10,91.24,231,18.00,19.00,20.00,19.00,78,33.77,354,1.48,8.16,87.76,0.73,0.02,0.83,1.02,4.00,3.00
2017,Paramount High,2772,Paramount Unified,Los Angeles,Paramount,33.90,-118.15,public,4794,1079,6.40,0.10,91.24,165,18.00,20.00,20.00,19.00,63,38.18,354,1.48,8.16,87.76,0.73,0.02,0.83,1.02,4.00,3.00
2018,Paramount High,2772,Paramount Unified,Los Angeles,Paramount,33.90,-118.15,public,4794,1112,6.40,0.10,91.24,80,20.00,21.00,21.00,21.00,36,45.00,354,1.48,8.16,87.76,0.73,0.02,0.83,1.02,4.00,3.00
2014,River Springs Charter,16951,Riverside County Office of Edu,Riverside,Temecula,33.50,-117.16,charter,5000,187,0.00,3.60,48.86,12,26.00,24.00,23.00,22.00,9,75.00,0,2.79,4.54,37.67,5.97,0.84,0.21,47.99,5.00,4.00
2015,River Springs Charter,16951,Riverside County Office of Education,Riverside,Temecula,33.50,-117.16,charter,5000,198,0.00,3.60,48.86,22,22.00,22.00,21.00,22.00,12,54.55,0,2.79,4.54,37.67,5.97,0.84,0.21,47.99,5.00,4.00


These are the schools with  high population. So these numbers are not outliers, they match with the real number of shool population.

In [45]:
final_df.columns

Index(['Sname', 'gsId', 'Dname', 'County', 'City', 'Lat', 'Lon', 'Type',
       'Enrollment', 'Enroll12', 'Percent Teachers InFirst SecondYear',
       'Plan504', 'FreeAnd Reduced PriceLunch', 'NumTstTakr', 'AvgScrEng',
       'AvgScrRead', 'AvgScrMath', 'AvgScrSci', 'NumGE21', 'PctGE21',
       'ActSat Number Participation', 'Asian', 'Black, non-Hispanic',
       'Hispanic', 'Multiracial', 'Native American or Native Alaskan',
       'Native Hawaiian or Other Pacific Islander', 'White, non-Hispanic',
       'gsRating', 'ParentRating'],
      dtype='object')

In [46]:
final_df.shape

(4218, 30)

In [47]:
#exporting data
final_df.to_csv('data.csv')